### Программа для сравнения отчета Splunk для инвойсов с данными BSS

### Запрос для выгрузки данных из BSS для сверки

SET @start_date = 20190201000000;
SET @end_date = 20190301000000;

SELECT
REP_DATE,Customer,USAGE_SOURCE
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
FROM
(
SELECT DATE_FORMAT(ev.TreatedTimestampUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'Mobile' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, TAP.MOC ev FORCE INDEX (ind_treated_moc)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc >= @start_date AND ev.TreatedTimestampUtc < @end_date
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE

UNION ALL

SELECT DATE_FORMAT(ev.TreatedTimestampUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'Mobile' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, TAP.MTC ev FORCE INDEX (ind_treated_mtc)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc >= @start_date AND ev.TreatedTimestampUtc < @end_date
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE

UNION ALL

SELECT DATE_FORMAT(ev.TreatedTimestampUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'Mobile' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, TAP.GPRS_CALL ev FORCE INDEX (ind_treated_gprs)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc >= @start_date AND ev.TreatedTimestampUtc < @end_date
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE
) t GROUP BY REP_DATE,Customer,USAGE_SOURCE

In [21]:
import pandas as pd
from os.path import join
import roamability as rb

downloads = r'c:\Users\balob\Downloads\DATA'
file_bss_mobile = 'BSS_MOBILE_REPORT.csv'
file_splunk_invoice = 'Monthly_auto_invoice_report-2019-06-01.csv'

In [23]:
# Загружаем данные BSS по Mobile

df_bss_mobile = pd.read_csv(join(downloads, file_bss_mobile), sep=';')
df_bss_mobile.drop('USAGE_SOURCE', axis=1, inplace=True)
df_bss_mobile.drop('REP_DATE', axis=1, inplace=True)
df_bss_mobile.Revenue = df_bss_mobile.Revenue.round().astype('int')
df_bss_mobile.head(3)

,Customer,Cost,Revenue
0,Cellact,6.198710,13
1,CELLULAR ABROAD,0.028592,0
2,Clay RT,153.011947,280


In [24]:
# Вызгрузить данныхе по Voip из BSS

sql_srt='''SELECT
DATE_FORMAT(ev.CallTimeUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'VOIP' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, voip.CALLS ev
WHERE ev.SponsImsi = s.SponsIMSI
AND ev.CallTimeUtc >= 20190501000000 AND ev.CallTimeUtc < 20190601000000
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE'''

# AND ev.CallTimeUtc >= 20190331220000 AND ev.CallTimeUtc < 20190430220000

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_voip = pd.read_sql_query(sql_srt, cnxn)

df_bss_voip.Revenue = df_bss_voip.Revenue.round() #.astype('int')
df_bss_voip.drop('USAGE_SOURCE', axis=1, inplace=True)
df_bss_voip.drop('REP_DATE', axis=1, inplace=True)
df_bss_voip.head(3)

,Customer,Cost,Revenue
0,Cellact,13.84355,39.0
1,Clay RT,45.80658,70.0
2,Dialoq - Genesisclub RT,0.01625,0.0


In [25]:
# Объединяем Mobile и Voip

df_bss = pd.merge(df_bss_mobile, df_bss_voip, how='outer', on='Customer', suffixes=['_bss_mobile', '_bss_voip'])
df_bss.head(3)

,Customer,Cost_bss_mobile,Revenue_bss_mobile,Cost_bss_voip,Revenue_bss_voip
0,Cellact,6.198710,13.0,13.84355,39.0
1,CELLULAR ABROAD,0.028592,0.0,NaN,NaN
2,Clay RT,153.011947,280.0,45.80658,70.0


In [26]:
# Загружаем данные Splunk

df_splunk_invoice = pd.read_csv(join(downloads, file_splunk_invoice))
df_splunk_invoice.head(3)

,Customer,Mobile Revenue,VoIP Revenue,Discount,Discount Amount,MonthlyFee,Total Invoice charge
0,CELLULAR ABROAD,0,0,0.0,-0.0,0,0.0
1,Cellact,13,39,0.0,-0.0,0,52.0
2,Clay RT,280,70,0.0,-0.0,0,350.0


In [27]:
df_bss.Revenue_bss_voip.sum()

11006.0

In [28]:
df_splunk_invoice['VoIP Revenue'].sum()

10985

In [29]:
# Делаем сравнительный отчет

df_compare = pd.merge(df_splunk_invoice, df_bss, how='outer', on='Customer')
df_compare.drop(['Discount', 'Discount Amount', 'MonthlyFee', 'Total Invoice charge', 'Cost_bss_mobile', 'Cost_bss_voip'],
                axis=1, inplace=True)

df_compare.fillna(0, inplace=True)
df_compare.head(3)

,Customer,Mobile Revenue,VoIP Revenue,Revenue_bss_mobile,Revenue_bss_voip
0,CELLULAR ABROAD,0.0,0.0,0.0,0.0
1,Cellact,13.0,39.0,13.0,39.0
2,Clay RT,280.0,70.0,280.0,70.0


In [30]:
# Несоответствия в Mobile

df_compare.loc[df_compare['Mobile Revenue'] != df_compare.Revenue_bss_mobile,
               ['Customer', 'Mobile Revenue', 'Revenue_bss_mobile']]

,Customer,Mobile Revenue,Revenue_bss_mobile
8,Joy telecom,8089.0,6917.0
11,MCN Telecom,131.0,132.0
12,MCN Telecom -- TESTING at 2018-07,3.0,2.0
36,Joy_telecom,0.0,1172.0


In [31]:
# Несоответствия в Voip

df_compare.loc[df_compare['VoIP Revenue'] != df_compare.Revenue_bss_voip,
              ['Customer', 'VoIP Revenue', 'Revenue_bss_voip']]

,Customer,VoIP Revenue,Revenue_bss_voip
6,HMG Int,2.0,0.0
7,HMG Int,0.0,2.0
20,Porto Seguro DMI RT,29.0,30.0
24,Roamability Test RT,24.0,33.0
25,Roamability Test SMART Sponsor RT,0.0,5.0
26,STI,0.0,1.0
29,Telzar 019 DMI RT,397.0,411.0
30,TinkLabs RT,10234.0,10221.0
31,Tottolli-Tech-Mobil,0.0,1.0
37,Roamability Test MB Sponsor RT,0.0,3.0
